In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install ambiance

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from matplotlib.animation import FuncAnimation
from matplotlib import rc
rc('animation', html='jshtml')

In [4]:
import sys
sys.path.append('/content/drive/Shareddrives/Aircraft Engine Project')

from Non_Dominated_Sorting import sort_vectors
from Real_Turbofan_Engine import TurboFanAnalysis
from Genetic_Algorithm import Genetic_Algorithm

In [20]:
engine = TurboFanAnalysis()

engine.setFlightMachNumber(1.0)
engine.setFlightConditions(12000)
engine.setFuelInfo(42.8E6)
engine.setInletOutletProperties()
engine.setBurnerProperties()
engine.setCompressorProperties(25)
engine.setFanProperties(1.5)
engine.setTurbineProperties(1600)
engine.setBypassRatio(5)

engine.initializeProblem()

In [21]:
# alpha, pi_c, pi_f, M_0, h, T_t4

max_values = np.array([10.0, 35.0, 5.0, 1.0, 15E3, 1780.0])
min_values = np.array([ 0.0,  1.0, 1.0, 0.5,  5E3,  800.0])
resolution = np.array([ 0.1,  0.1, 0.1, 0.1,  1E2,   10.0])

num_bits = np.array( np.ceil( np.log2( (max_values - min_values) / resolution ) ), dtype=int) + 1
bits_conversion_array = [np.power(2, np.arange(bits)) for bits in num_bits]

In [22]:
def bits_to_input(bits) :

  i = 0
  j = num_bits[0]
  alpha = min_values[0] + (max_values[0] - min_values[0]) * np.sum(bits[:, i:j] * bits_conversion_array[0], axis=1) / (2**num_bits[0] - 1)

  i = j
  j += num_bits[1]
  pi_c = min_values[1] + (max_values[1] - min_values[1]) * np.sum(bits[:, i:j] * bits_conversion_array[1], axis=1) / (2**num_bits[1] - 1)

  i = j
  j += num_bits[2]
  pi_f = min_values[2] + (max_values[2] - min_values[2]) * np.sum(bits[:, i:j] * bits_conversion_array[2], axis=1) / (2**num_bits[2] - 1)

  i = j
  j += num_bits[3]
  M_0 = min_values[3] + (max_values[3] - min_values[3]) * np.sum(bits[:, i:j] * bits_conversion_array[3], axis=1) / (2**num_bits[3] - 1)

  i = j
  j += num_bits[4]
  h = min_values[4] + (max_values[4] - min_values[4]) * np.sum(bits[:, i:j] * bits_conversion_array[4], axis=1) / (2**num_bits[4] - 1)

  i = j
  j += num_bits[5]
  T_t4 = min_values[5] + (max_values[5] - min_values[5]) * np.sum(bits[:, i:j] * bits_conversion_array[5], axis=1) / (2**num_bits[5] - 1)

  return np.array([alpha, pi_c, pi_f, M_0, h, T_t4])

def analyseEngine(alpha, pi_c, pi_f, M_0, h, T_t4) :
  
  engine.setBypassRatio(alpha)
  engine.setCompressorProperties(pi_c)
  engine.setFanProperties(pi_f)
  engine.setFlightMachNumber(M_0)
  engine.setFlightConditions(h)
  engine.setTurbineProperties(T_t4)

  engine.performAnalysis()

  pass

In [23]:
def engineFitnessFunction(bits) :

  alpha, pi_c, pi_f, M_0, h, T_t4 = bits_to_input(bits)
  
  analyseEngine(alpha, pi_c, pi_f, M_0, h, T_t4)

  return np.column_stack((engine.getSpecificThrust(), - engine.getThrustSpecificFuelConsumtion()))

In [28]:
problem = Genetic_Algorithm()

problem.set_dimensions(sum(num_bits), 2)
problem.set_number_of_chromosomes(1000)
problem.set_probabilities(0.8, 0.3)
problem.set_fitness_function(engineFitnessFunction)
problem.set_sorting_function(sort_vectors)

problem.begin()

Pareto_fronts = []
Other_chromosomes = []

for i in range(50) :
    
    problem.iterate()
    Pareto_fronts.append(problem.get_Pareto_Front_fitness_values())
    Other_chromosomes.append(problem.get_non_Pareto_Front_fitness_values())

problem.stop_iterations()

/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:344: RuntimeWarning: invalid value encountered in float_power
  self._pi_t  = np.float_power(self._tau_t, self._gamma_t / ((self._gamma_t - 1.0) * self._e_t))
/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:360: RuntimeWarning: invalid value encountered in sqrt
  (np.float_power(product_pi, (self._gamma_t - 1.0) / self._gamma_t) - 1.0)
/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:12: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(gamma * gas_constant * temperature)


In [27]:
fig, ax = plt.subplots(figsize=(12,8))

# manager = plt.get_current_fig_manager()
# manager.window.showMaximized()

points, = ax.plot(  Pareto_fronts[0][:, 0], 
                    -Pareto_fronts[0][:, 1],
                    'bo', ms=5, color='blue',
                    label='Points on Pareto Front')

other_points, = ax.plot(    Other_chromosomes[0][:, 0],
                            -Other_chromosomes[0][:, 1],
                            'bo', ms=3, color='red',
                            label='Points not on Pareto Front')

# set axes labels
ax.set_xlabel(r'Specific Thrust (in $\frac{N}{kg/s}$)', fontsize=15)
ax.set_ylabel(r'Thrust Specific Fuel Consumption (in $\frac{kg/s}{N}$)', fontsize=15)

ax.set_ylim(bottom=0)

# set title
ax.set_title('Pareto Front at Generation # 0', fontsize=15)

# Grid
ax.minorticks_on()
ax.grid(which='minor', ls='--', c='green', alpha=0.5)

ax.grid(which='major', c='grey', alpha=0.5)

# Legend
ax.legend(fontsize=15, loc=2)

def animate(i) :

    # set title
    ax.set_title('Pareto Front at Generation # ' + str(i+1), fontsize=15)

    # print(Pareto_fronts[i])
    points.set_data(Pareto_fronts[i][:, 0], -Pareto_fronts[i][:, 1])
    other_points.set_data(Other_chromosomes[i][:, 0], -Other_chromosomes[i][:, 1])
    # points.set_markersize(10)

    return points, other_points, 

anim = FuncAnimation(fig, animate, frames=50, interval=5E3/50)
anim

Output hidden; open in https://colab.research.google.com to view.

In [16]:
best_genes = problem.get_Pareto_Front_chromosomes()

alpha, pi_c, pi_f, M_0, h, T_t4 = bits_to_input(best_genes)

analyseEngine(alpha, pi_c, pi_f, M_0, h, T_t4)

/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:344: RuntimeWarning: invalid value encountered in float_power
  self._pi_t  = np.float_power(self._tau_t, self._gamma_t / ((self._gamma_t - 1.0) * self._e_t))
/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:360: RuntimeWarning: invalid value encountered in sqrt
  (np.float_power(product_pi, (self._gamma_t - 1.0) / self._gamma_t) - 1.0)
/content/drive/Shareddrives/Aircraft Engine Project/Real_Turbofan_Engine.py:12: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(gamma * gas_constant * temperature)


In [17]:
specific_thrust = engine.getSpecificThrust()
TSFC = engine.getThrustSpecificFuelConsumtion()

In [18]:
# air = Real_Turbofan_Engine.ambiance.Atmosphere(h)
args = np.argwhere(np.invert(np.isnan(TSFC)))

TSFC = TSFC[args]
specific_thrust = specific_thrust[args]

m_dot_0 = 1.0 * 19.34 * M_0[args] * 340
best_arg = np.argmin(
    TSFC[specific_thrust * m_dot_0 >= 40000]
)

In [19]:
print(TSFC[best_arg])

[1.06963239e-05]


In [ ]:
TSFC

array([[ 2.66716836e-05],
       [ 2.43379290e-05],
       [ 1.51197399e-05],
       [ 6.91659988e-06],
       [ 2.06200757e-05],
       [ 1.64148099e-05],
       [ 3.20988792e-05],
       [ 3.90734575e-05],
       [ 1.46796210e-05],
       [ 1.57394315e-05],
       [ 1.38036372e-05],
       [ 1.43490676e-05],
       [ 2.10682316e-05],
       [ 1.66861069e-05],
       [ 1.79662796e-05],
       [ 3.15398206e-05],
       [ 1.99835168e-05],
       [ 2.20656075e-05],
       [ 5.81883823e-06],
       [ 1.96451875e-05],
       [-1.63453420e-04],
       [ 1.46627203e-05],
       [ 3.54992429e-05],
       [ 1.89336542e-05],
       [ 2.05325509e-05],
       [ 2.89930623e-05],
       [ 3.80892030e-05],
       [ 2.14316034e-05],
       [ 2.41504883e-05],
       [ 1.85653590e-05],
       [ 2.58817828e-05],
       [ 2.89715601e-05],
       [ 2.17961370e-05],
       [ 7.52415440e-06],
       [ 3.25879690e-05],
       [ 7.89577760e-06],
       [ 1.67993959e-05],
       [ 3.24207845e-05],
       [ 8.8